In [1]:
import os
import numpy as np
import pandas as pd
import mne
import scipy.io
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import FastICA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier



In [2]:
#filePath= '/Users/yaman/Library/CloudStorage/GoogleDrive-s203192@stn.nagaokaut.ac.jp/マイドライブ/実験データ/20230519_zikken'
filePath= '/Users/yaman/Library/CloudStorage/GoogleDrive-s203192@stn.nagaokaut.ac.jp/マイドライブ/実験データ/20230529_1_zikken'
#filePath= '/Users/yaman/Library/CloudStorage/GoogleDrive-s203192@stn.nagaokaut.ac.jp/マイドライブ/実験データ/20230529_2_zikken'
# filePath = 'G:/マイドライブ/実験データ/20230519_zikken'
brainAdress = ['Fp1','AF7','AF3','F1','F3','F5','F7','FT7','FC5',
'FC3','FC1','C1','C3','C5','T7','TP7','CP5','CP3','CP1','P1','P3',
'P5','P7','P9','PO7','PO3','O1','Iz','Oz','POz','Pz','CPz','Fpz','Fp2',
'AF8','AF4','AFz','Fz','F2','F4','F6','F8','FT8','FC6','FC4','FC2',
'FCz','Cz','C2','C4','C6','T8','TP8','CP6','CP4','CP2','P2','P4',
'P6','P8','P10','PO8','PO4','O2']



motor_brainAdress = [9,10,11,12,13,17,18,31,44,45,46,47,48,49,50,54,55]




# dataクラス
class O_Data:
    def __init__(self, eeg, index, label):
        self.eeg = eeg
        self.index = index
        self.label = label
        self.stack = []

    def push(self, item):
        self.stack.append(item)

o_data65_label1 = list()#label1のデータ
trial_num = list()#label1のデータ

o_data = list()
label_data = list()#labelを結合したデータ


All_data = list()  # 全データを結合

def Standardization(data): #標準化
    after_data = scipy.stats.zscore(data)
    #b = np.average(after_data)
    #c = np.var(after_data)
    return after_data

file_name = list()  # すべての.matファイルの名前
for file in os.listdir(filePath):
    base, ext = os.path.splitext(file)
    if ext == '.mat':
        file_name = sorted(file_name)
        file_name.append(file)
print('Filelist')
print(file_name)

Filelist
['EEG_grasping_29-May-2023_sub1-1.mat', 'EEG_grasping_29-May-2023_sub1-2.mat', 'EEG_grasping_29-May-2023_sub2-1.mat', 'EEG_grasping_29-May-2023_sub2-2.mat', 'EEG_grasping_29-May-2023_sub2-3.mat', 'EEG_grasping_29-May-2023_sub2-4.mat', 'EEG_grasping_29-May-2023_sub3-1.mat', 'EEG_grasping_29-May-2023_sub3-2.mat', 'EEG_grasping_29-May-2023_sub3-3.mat', 'EEG_grasping_29-May-2023_sub3-4.mat', 'EEG_grasping_29-May-2023_sub3-5.mat', 'EEG_grasping_29-May-2023_sub3-6.mat']


In [3]:
#解析するデータを選択
file_name1 = [file_name[6],file_name[7],file_name[8],file_name[9],file_name[10],file_name[11]]
file_num1 = len(file_name1)
trial_num = 30
all_trial = len(file_name1) * trial_num
all_d = 0
CH = 64
#label1を読み込む
for s in range(file_num1):
    Dictionary = scipy.io.loadmat(filePath+"/"+file_name1[s])
    for t in range(trial_num):
        for i, key in enumerate(Dictionary.keys()):
            if i > 2:
                a = Dictionary[key]
                b = a[0, t][0][0]
                eeg = b[0][0:66, :]
                index = b[1]
                samplerate = b[2]
                label = b[5][0, 0]
                o_data = O_Data(eeg, index, label)
                o_data65_label1.append(o_data)
                
print("image data_loading")
print(file_name1)

image data_loading
['EEG_grasping_29-May-2023_sub3-1.mat', 'EEG_grasping_29-May-2023_sub3-2.mat', 'EEG_grasping_29-May-2023_sub3-3.mat', 'EEG_grasping_29-May-2023_sub3-4.mat', 'EEG_grasping_29-May-2023_sub3-5.mat', 'EEG_grasping_29-May-2023_sub3-6.mat']


In [4]:
c1 = o_data65_label1[0].eeg[: , 0:10240]
for n in range(len(file_name1)*trial_num-1):  # all_trial
    g1 = o_data65_label1[n+1].eeg[: , 0:10240]
    c1 = np.dstack([c1, g1])


In [5]:
all_data = c1.transpose(2,0,1)
print(np.shape(all_data))
all_data = all_data.astype(np.double)

(180, 66, 10240)


In [6]:
filter_bandpass = signal.firwin(numtaps=51,cutoff=[6,40],fs = 1024,pass_zero=False)

In [7]:
for n in range(3):
    for j in range(60):
        for k in range(66):
            all_data[n*60+j][k]= abs(all_data[n*60+j][k]-np.mean(all_data[n*60+j][0:64],axis=0))
            all_data[n*60+j][k] = signal.lfilter(filter_bandpass,1,all_data[n*60+j][k])
all_data = scipy.stats.zscore(all_data)

In [8]:
label_data = list()#labelを結合したデータ
for n in range(all_trial):
    label_data.append(o_data65_label1[n].label)
print(label_data)
print(np.shape(label_data))

[1, 2, 2, 1, 1, 2, 1, 3, 2, 3, 3, 1, 3, 1, 2, 2, 2, 1, 3, 2, 2, 3, 1, 2, 3, 3, 3, 1, 1, 3, 1, 1, 3, 2, 3, 2, 3, 3, 1, 3, 3, 3, 1, 2, 1, 2, 2, 3, 2, 2, 3, 1, 1, 3, 2, 2, 1, 1, 1, 2, 2, 3, 2, 1, 2, 3, 1, 2, 2, 2, 3, 3, 1, 2, 1, 3, 2, 3, 1, 2, 1, 3, 1, 3, 1, 1, 2, 3, 3, 1, 3, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 3, 2, 2, 3, 2, 1, 3, 2, 3, 3, 1, 1, 3, 3, 2, 3, 3, 1, 2, 2, 1, 1, 3, 3, 2, 3, 1, 2, 1, 3, 2, 2, 3, 3, 1, 3, 1, 2, 2, 1, 1, 1, 3, 3, 1, 2, 3, 2, 2, 1, 2, 1, 2, 2, 3, 2, 1, 3, 1, 3, 2, 1, 3, 3, 3, 2, 1, 1, 2, 3, 2, 3, 1, 1, 3, 2, 2, 3, 1]
(180,)


In [9]:
#numpy配列の定義
task1 = o_data65_label1[0].eeg[: , 0:10240]
task2 = o_data65_label1[0].eeg[: , 0:10240]
task3 = o_data65_label1[0].eeg[: , 0:10240]
real_trial_num =list()
for n in range(all_trial):
    if label_data[n] == 1:
        join = o_data65_label1[n].eeg[: , 0:10240]
        task1 = np.dstack([task1, join])
    if label_data[n] == 2:
        join = o_data65_label1[n].eeg[: , 0:10240]
        task2 = np.dstack([task2, join])
    if label_data[n] == 3:
        join = o_data65_label1[n].eeg[: , 0:10240]
        task3 = np.dstack([task3, join])
#task1低速度
task1_data = task1.transpose(2,0,1)
task1_data = task1_data.astype(np.double)
task1_data = np.delete(task1_data,0,axis=0)
print(np.shape(task1_data))
#task2低速度
task2_data = task2.transpose(2,0,1)
task2_data = task2_data.astype(np.double)
task2_data = np.delete(task2_data,0,axis=0)
print(np.shape(task2_data))
#task3想起なし
task3_data = task3.transpose(2,0,1)
task3_data = task3_data.astype(np.double)
task3_data = np.delete(task3_data,0,axis=0)
print(np.shape(task2_data))

AttributeError: 'list' object has no attribute 'label'

In [ ]:
# processed_task1 = task1_data[:,range(64),1024*8:1024*10]
# processed_task2 = task2_data[:,range(64),1024*8:1024*10]
# processed_task3 = task3_data[:,range(64),1024*8:1024*10]

# csp_train1 = task1_data[:,range(64),1024*4:1024*8]
# csp_train2 = task2_data[:,range(64),1024*4:1024*8]
# csp_train3 = task3_data[:,range(64),1024*4:1024*8]

processed_task1 = task1_data[:,range(64),1024*4:1024*10]
processed_task2 = task2_data[:,range(64),1024*4:1024*10]
processed_task3 = task3_data[:,range(64),1024*4:1024*10]

csp_train1 = task1_data[:,range(64),1024*0:1024*4]
csp_train2 = task2_data[:,range(64),1024*0:1024*4]
csp_train3 = task3_data[:,range(64),1024*0:1024*4]

In [ ]:
class_data = np.concatenate([processed_task1,processed_task2])
csp_class_data = np.concatenate([csp_train1,csp_train2])
class_label = np.zeros(120,dtype=int)
class_label[60:] = 1

In [ ]:
import random
all_indices = list(range(120))
random.shuffle(all_indices)
csp_class_data2 = csp_class_data[all_indices]
class_label2 = class_label[all_indices]

In [ ]:
# class_data_reshape = np.reshape(class_data,(120,64*1024*2))
# X_train, X_test, Y_train, Y_test = train_test_split(class_data_reshape,class_label, test_size=0.25,random_state=42)
# model =SVC(kernel='linear')
# model.fit(X_train,Y_train)
# print(model.score(X_test, Y_test))

In [ ]:
# Assemble a classifier
lda = LinearDiscriminantAnalysis()
#csp = CSP(n_components=2, log=None, reg=None)
csp = CSP(n_components=4, reg=None, norm_trace=False, transform_into='average_power',cov_est='epoch')
# Use scikit-learn Pipeline with cross_val_score function
n_channels = 64
sampling_freq = 1024  # in Hertz
info = mne.create_info(ch_names=brainAdress, ch_types="eeg", sfreq=sampling_freq)
info.set_montage('biosemi64')

In [ ]:
# Printing the results
class_balance = np.mean(class_label == class_label[0])
class_balance = max(class_balance, 1. - class_balance)
#print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),class_balance))

# plot CSP patterns estimated on full data for visualization
kenshou = csp.fit_transform(csp_class_data, class_label)
csp.fit_transform(csp_class_data, class_label)
csp.plot_patterns(info,ch_type='eeg', units='None', size=1.5)

In [ ]:
print(np.shape(kenshou))

In [ ]:
np.shape(class_data)

In [ ]:
# class_datafit = np.reshape(class_data,(120,64*1024*6))
# X_train_none, X_test_none, Y_train_none, Y_test_none = train_test_split(class_datafit,class_label, test_size=0.25,random_state=42)
# model =SVC(kernel='linear')
# model.fit(X_train_none,Y_train_none)
# print(model.score(X_test_none, Y_test_none))

In [ ]:
class_data_cspfit = csp.transform(class_data)
# class_data_cspfit = np.reshape(class_data_cspfit,(84,4*6144))
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(class_data_cspfit,class_label, test_size=0.25,random_state=42)
model =SVC(kernel='linear')
model.fit(X_train2,Y_train2)
print(model.score(X_test2, Y_test2))

In [ ]:
params = {
    "C":np.logspace(-1,10,12),
    "kernel":["rbf", "sigmoid"],
    "gamma":np.logspace(-8,3,12)
}
grid = GridSearchCV(model, params,scoring="accuracy", cv=5)

grid.fit(X_train2, Y_train2)

print("grid_best_score",grid.best_score_)
print(grid.best_params_)

pred = grid.predict(X_test2)
print(classification_report(Y_test2, pred))

In [ ]:
from sklearn.decomposition import PCA

#PCAを用意
pca = PCA(n_components=2)
#PCAで次元圧縮
pca.fit(X_train2)
#PCAの結果を元にデータを変換
pca_X = pca.transform(X_train2)

In [ ]:
#SVNデータの点を表示
Z = grid.predict(X_train2)

plt.scatter(pca_X[:,0], pca_X[:,1], c=Z)
plt.xlabel("x1")
plt.ylabel("x2")

In [ ]:
plt.show()